In [0]:
import numpy as np
import pandas as pd

In [34]:
!pip install pydub

In [0]:
import pydub

In [0]:
from keras.models import Sequential,load_model
from keras.layers import Dense,LSTM,LeakyReLU
from scipy.io.wavfile import read,write

In [37]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
root_path = '/content/gdrive/My Drive/data_music/'

In [39]:
!ls '/content/gdrive/My Drive/data_music/'

Eminem.mp3  Numb_piano.mp3


In [40]:
sound=pydub.AudioSegment.from_mp3('/content/gdrive/My Drive/data_music/Numb_piano.mp3')
sound.export("Numb.wav",format="wav")
sound=pydub.AudioSegment.from_mp3('/content/gdrive/My Drive/data_music/Eminem.mp3')
sound.export('Eminem.wav',format='wav')


<_io.BufferedRandom name='Eminem.wav'>

In [0]:
rate,music1=read('Numb.wav')

In [0]:
rate,music2=read('Eminem.wav')

In [43]:
music2

array([[ 0,  0],
       [ 0,  0],
       [ 0,  0],
       ...,
       [-1,  1],
       [ 0,  0],
       [ 0,  0]], dtype=int16)

In [0]:
music1=pd.DataFrame(music1[0:400000,:])
music2=pd.DataFrame(music2[0:40000,:])

In [45]:
music1.head()

,0,1
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0


In [46]:
music1.tail()

,0,1
399995,-1325,-138
399996,-1310,-151
399997,-1280,-163
399998,-1245,-188
399999,-1206,-223


In [0]:
def create_train_dataset(df, look_back):
    dataX1, dataX2 , dataY1 , dataY2 = [],[],[],[]
    for i in range(len(df)-look_back-1):
        dataX1.append(df.iloc[i : i + look_back, 0].values)
        dataX2.append(df.iloc[i : i + look_back, 1].values)
        dataY1.append(df.iloc[i + look_back, 0])
        dataY2.append(df.iloc[i + look_back, 1])
    return np.array(dataX1), np.array(dataX2), np.array(dataY1), np.array(dataY2)

In [0]:

X1 , X2, y1 , y2 = create_train_dataset(pd.concat([music1.iloc[0 : 160000, :],music2.iloc[0 : 160000, :]], axis=0), 3)

In [0]:

# function to create train data by shifting the music data
def create_test_data(df, look_back):
    dataX1, dataX2 = [], []
    for i in range(len(df)-look_back-1):
        dataX1.append(df.iloc[i : i + look_back, 0].values)
        dataX2.append(df.iloc[i : i + look_back, 1].values)
        
    return np.array(dataX1), np.array(dataX2)

In [0]:
test1, test2 = create_test_data(pd.concat([music1.iloc[160001 : 400000, :],music2.iloc[160001 : 400000, :]], axis=0), 3)

In [0]:
X1 = X1.reshape((-1, 1, 3))
X2 = X2.reshape((-1, 1, 3))

In [53]:


# LSTM Model for channel 1 of the music data
model = Sequential()
model.add(LSTM(units=100, activation='linear', input_shape=(None, 3)))
model.add(LeakyReLU())
model.add(Dense(units=50, activation='linear'))
model.add(LeakyReLU())
model.add(Dense(units=25, activation='linear'))
model.add(LeakyReLU())
model.add(Dense(units=12, activation='linear'))
model.add(LeakyReLU())
model.add(Dense(units=1, activation='linear'))
model.add(LeakyReLU())

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X1, y1, epochs=20, batch_size=100)






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/20





199996/199996 [==============================] - 13s 67us/step - loss: 39781.0893
Epoch 2/20
199996/199996 [==============================] - 12s 62us/step - loss: 4627.9993
Epoch 3/20
199996/199996 [==============================] - 12s 59us/step - loss: 3959.5701
Epoch 4/20
199996/199996 [==============================] - 12s 59us/step - loss: 3381.2332
Epoch 5/20
199996/199996 [==============================] - 12s 58us/step - loss: 3419.0472
Epoch 6/20
199996/199996 [==============================] - 11s 56us/step - loss: 2793.3496
Epoch 7/20
199996/199996 [==============================] - 12s 58us/step - loss: 2680.2665
Epoch 8/20
199996/199996 [==============================] - 12s 58us/step - loss: 2563.7819
Epoch 9/20
199996/199996 [==============================] - 12s 61us/step - loss: 2565.2919
Epoch 10/20
199996/199996 [==============================] - 12s 

In [54]:
model2 = Sequential()
model2.add(LSTM(units=100, activation='linear', input_shape=(None, 3)))
model2.add(LeakyReLU())
model2.add(Dense(units=50, activation='linear'))
model2.add(LeakyReLU())
model2.add(Dense(units=25, activation='linear'))
model2.add(LeakyReLU())
model2.add(Dense(units=12, activation='linear'))
model2.add(LeakyReLU())
model2.add(Dense(units=1, activation='linear'))
model2.add(LeakyReLU())

model2.compile(optimizer='adam', loss='mean_squared_error')
model2.fit(X2, y2, epochs=20, batch_size=100)

Epoch 1/20
199996/199996 [==============================] - 13s 66us/step - loss: 116462.7582
Epoch 2/20
199996/199996 [==============================] - 12s 60us/step - loss: 8892.9890
Epoch 3/20
199996/199996 [==============================] - 12s 60us/step - loss: 7022.0034
Epoch 4/20
199996/199996 [==============================] - 12s 60us/step - loss: 7193.8751
Epoch 5/20
199996/199996 [==============================] - 12s 60us/step - loss: 6041.8320
Epoch 6/20
199996/199996 [==============================] - 12s 60us/step - loss: 5379.6579
Epoch 7/20
199996/199996 [==============================] - 12s 59us/step - loss: 5351.7845
Epoch 8/20
199996/199996 [==============================] - 11s 57us/step - loss: 5162.0254
Epoch 9/20
199996/199996 [==============================] - 12s 58us/step - loss: 4219.0209
Epoch 10/20
199996/199996 [==============================] - 12s 60us/step - loss: 4223.1387
Epoch 11/20
199996/199996 [==============================] - 12s 59us/step - 

In [55]:
model.model.save('lstm1.h5')
model2.model.save('lstm2.h5')

/usr/local/lib/python3.6/dist-packages/keras/engine/sequential.py:111: UserWarning: `Sequential.model` is deprecated. `Sequential` is a subclass of `Model`, you can just use your `Sequential` instance directly.
  warnings.warn('`Sequential.model` is deprecated. '


In [0]:

# loading the saved models
lstm1 = load_model('lstm1.h5')
lstm2 = load_model('lstm2.h5')

In [0]:
pred_lstm1 = lstm1.predict(test1.reshape(-1, 1, 3))
pred_lstm2 = lstm2.predict(test2.reshape(-1, 1, 3))



In [0]:
write('pred_lstm.wav', rate, pd.concat([pd.DataFrame(pred_lstm1.astype('int16')), pd.DataFrame(pred_lstm2.astype('int16'))], axis=1).values)

In [0]:
write('original.wav',rate, pd.concat([music1.iloc[160001 : 400000, :], music2.iloc[160001 : 400000, :]], axis=0).values)

In [0]:
from IPython.display import Audio

In [72]:
Audio("pred_lstm.wav")

In [73]:
Audio("original.wav")